# Classifying Diabetic Retinopathy using Decision Trees
---

#### Attribute Information:
- The binary result of quality assessment:
    0 = bad quality 1 = sufficient quality.
- You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

In [2]:
import pandas as pd
from math import log2

# GLOBAL VARIABLES
CLASS_LABEL = 19
ONE_HUNDRED = 100.0

In [1]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    
    def __init__(self, label, features):
        self.label = label                  # the classification label of this data point
        self.features = features            # a list of feature values for this data point

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [3]:
def get_data(filename):
    data = []

    messidor_df = pd.read_csv(filename, delimiter=',', header=None)

    # iterate over rows using itterrows() and create DataPoint objects
    for index, row in messidor_df.iterrows():
        label = row.iloc[CLASS_LABEL]
        features = row.iloc[:-1].tolist() # exclude last column (the class label)
        data_point = DataPoint(label, features)
        data.append(data_point)
    
    return data

In [4]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

In [5]:
# Recursive implementation
def make_prediction(tree_root, data_point):
    # base case
    if tree_root.is_leaf:
        return tree_root.prediction
    
    # otherwise, lets check the feature value
    feature_value = data_point.features[tree_root.feature_idx]

    # now we split depending on the tree's threshold value
    if feature_value < tree_root.thresh_val:
        return make_prediction(tree_root.left_child, data_point)
    else:
        return make_prediction(tree_root.right_child, data_point)

In [6]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []

    for data_point in data:
        feature_value = data_point.features[feature_idx]
        if feature_value < threshold:
            left_split.append(data_point)
        else:
            right_split.append(data_point)
        
    return (left_split, right_split)

\begin{equation*}
Entropy(data) = \sum_{n=1} ^{c} -p_n \log_2 p_n
\end{equation*}

In [7]:
def calc_entropy(data):
    positive_cases = 0
    negative_cases = 0
    total_cases = 0

    # first calculate the frequency for each class label in the data
    for data_point in data:
        if data_point.label == 1:
            positive_cases += 1
        else:
            negative_cases += 1
        total_cases += 1

    # now we can calculate entropy for each label
    entropy = 0.0
    pos_calc = 0
    neg_calc = 0
    if positive_cases == 0:
        pos_calc = 0
    elif negative_cases == 0:
        neg_calc = 0
    else:
        pos_probability = positive_cases / total_cases
        neg_probability = negative_cases / total_cases
        pos_calc = pos_probability * log2(pos_probability)
        neg_calc = neg_probability * log2(neg_probability)

    entropy = -(pos_calc) - (neg_calc)
    return entropy

\begin{equation*}
Gain_{split} = Impurity(parent) - (\sum_{n=1} ^{k} w_n Impurity(n))
\end{equation*}

In [8]:
def calc_best_threshold(data, feature_idx):
 
    # custom function to calc gain split using entropy
    def calc_gain_split(data, feature_idx, threshold):
        parent_entropy = calc_entropy(data)
        left_split, right_split = split_dataset(data, feature_idx, threshold)
        left_entropy = calc_entropy(left_split)
        right_entropy = calc_entropy(right_split)
        left_weight = len(left_split) / len(data)
        right_weight = len(right_split) / len(data)
        return parent_entropy - ((left_weight * left_entropy) + (right_weight * right_entropy))

    best_gain_split = -1
    best_thresh = -1
    feature_set = set()
    
    # obtain set of unique features from given DataPoint
    for data_point in data:
        feature_set.add(data_point.features[feature_idx])
    ordered_features = sorted(feature_set)
    
    # __GAIN SPLIT AND THRESHOLD CALCULATIONS__
    for i in range(len(ordered_features) - 1):
        cur_val = ordered_features[i]
        next_val = ordered_features[i + 1]
        threshold = (cur_val + next_val) / 2.0
        gain_split = calc_gain_split(data, feature_idx, threshold)
        
        # update best_gain_split and best_threshold if needed
        if gain_split > best_gain_split:
            best_gain_split = gain_split
            best_thresh = threshold

    return (best_gain_split, best_thresh)

In [9]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
    best_gain = 0
    num_features = len(data[0].features)
    
    # now lets iterate through each feature from our data and see which feature give us the best total gain
    for feature_idx in range(num_features):
        gain, threshold = calc_best_threshold(data, feature_idx)
        # update best_gain if needed 
        if gain > best_gain:
            best_gain = gain
            best_feature = feature_idx
            best_thresh = threshold
            
    return (best_feature, best_thresh)

In [10]:
def create_leaf_node(data):
    leaf_node = TreeNode()
    leaf_node.is_leaf = True

    positive_cases = 0
    negative_cases = 0

    # first calculate the frequency for each class label in the data
    for data_point in data:
        if data_point.label == 1:
            positive_cases += 1
        else:
            negative_cases += 1

    # Special case
    if positive_cases == negative_cases:
        leaf_node.prediction = 1
    elif positive_cases > negative_cases:
        leaf_node.prediction = 1
    else:
        leaf_node.prediction = 0
        
    return leaf_node

In [11]:
def create_decision_tree(data, max_levels):
    # base case
    if max_levels <= 1:
        return create_leaf_node(data)

    feature_idx, threshold = identify_best_split(data)
    
    # special case: no impurity (entropy = 0)
    if feature_idx == None:
        return create_leaf_node(data)

    # create an internal node at this point in the recursion
    internal_node = TreeNode()
    internal_node.feature_idx = feature_idx
    internal_node.thresh_val = threshold
    internal_node.is_leaf = False

    # create internal node's children and then recurse
    left_data, right_data = split_dataset(data, feature_idx, threshold)
    internal_node.left_child = create_decision_tree(left_data, max_levels - 1)
    internal_node.right_child = create_decision_tree(right_data, max_levels - 1)
    
    return internal_node

In [12]:
def calc_accuracy(tree_root, test_data):
    correct_predictions = 0
    for data_point in test_data:
        actual_label = data_point.label
        predicted_label = make_prediction(tree_root, data_point)
        if actual_label == predicted_label:
            correct_predictions += 1
    
    return correct_predictions / len(test_data)

In [14]:
import time
filename = 'messidor_data.txt'
data = get_data(filename)

# partition data into train_set and test_set
k_folds = 5
fold_size = len(data) // k_folds
total_fold_accuracies = 0.0

# test model 
for i in range(k_folds):
    start_idx = i * fold_size
    end_idx = (i + 1) * fold_size 
    train_set = data[:start_idx] + data[end_idx:]
    test_set = data[start_idx:end_idx]

    print('Training set size:', len(train_set))
    print('Test set size    :', len(test_set))

    start = time.time()

    # create the decision tree
    tree = create_decision_tree(train_set, 10)

    end = time.time()
    print ('Time taken:', end - start)

    # calculate the accuracy of the tree
    accuracy = calc_accuracy(tree, test_set)
    total_fold_accuracies += accuracy
    print('The accuracy on the test set is ', str(accuracy * ONE_HUNDRED))
    print()

avg_accuracy = (total_fold_accuracies / k_folds) * ONE_HUNDRED
print(f'The Average Accuracy on a 5-fold Cross Validation: {avg_accuracy}\n',)
#tree.printTree() # for debugging purposes

Training set size: 920
Test set size    : 230
Time taken: 3.0063350200653076
The accuracy on the test set is  63.91304347826087

Training set size: 920
Test set size    : 230
Time taken: 2.7375922203063965
The accuracy on the test set is  63.04347826086957

Training set size: 920
Test set size    : 230
Time taken: 2.7224597930908203
The accuracy on the test set is  66.95652173913044

Training set size: 920
Test set size    : 230
Time taken: 2.839056968688965
The accuracy on the test set is  63.91304347826087

Training set size: 920
Test set size    : 230
Time taken: 2.7841410636901855
The accuracy on the test set is  64.34782608695652

The Average Accuracy on a 5-fold Cross Validation: 64.43478260869566

